In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
file=open('data_nwp.txt', 'r', encoding = "utf8")

In [3]:
lines=[]
for i in file:
    lines.append(i)

In [4]:
data=""
for i in lines:
    data=' '.join(lines)

In [6]:
data=data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [7]:
data=data.split()
data=' '.join(data)

In [ ]:
data[: 1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])

In [ ]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [ ]:
sequence_data=tokenizer.texts_to_sequences([data])[0]

In [ ]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [ ]:
len(sequence_data)

105879

In [ ]:
vocab_size = len(tokenizer.word_index)+1

In [ ]:
vocab_size

8200

In [ ]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [ ]:
len(sequence)

105876

In [ ]:
sequence=np.array(sequence)

In [ ]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [ ]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
print(X)

[[   1 1561    5]
 [1561    5  129]
 [   5  129   34]
 ...
 [  28    1 8198]
 [   1 8198 8199]
 [8198 8199 3187]]


In [ ]:
print(y)

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
y[: 5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_shape=(3,)))  # input_shape=(3,) for 3 tokens in each sequence
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))


In [ ]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 3, 10)               │          82,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ (None, 3, 1000)             │       4,044,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 1000)                │       8,004,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 8200)                │       8,208,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,339,200 (81.40 MB)

 Trainable params: 21,339,200 (81.40 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("next_words.keras", monitor="loss", verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=2, batch_size=64, callbacks=[checkpoint])


Epoch 1/2
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - loss: 6.6613
Epoch 1: loss improved from inf to 6.34356, saving model to next_words.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 1107s 666ms/step - loss: 6.6611
Epoch 2/2
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 665ms/step - loss: 5.7861
Epoch 2: loss improved from 6.34356 to 5.76273, saving model to next_words.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 1168s 670ms/step - loss: 5.7861


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model with the .keras extension
model = load_model('next_words.keras')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_word(model, tokenizer, text):
    # Convert the input text to a sequence and pad/truncate to match input length (assuming 3 words are needed)
    sequence = tokenizer.texts_to_sequences([' '.join(text)])  # join text list into a string
    sequence = np.array(sequence)

    # Get the prediction
    preds = np.argmax(model.predict(sequence), axis=-1)
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    print(predicted_word)
    return predicted_word

while True:
    text = input("Enter your line: ")
    if text == "1":
        break
    else:
        # Take only the last three words of the input
        text = text.split(" ")
        text = text[-3:]  # get the last 3 words as a list
        predict_word(model, tokenizer, text)


Enter your line: Here we go
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 513ms/step
to
Enter your line: When i used to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
the
Enter your line: Can I
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
have
Enter your line: Once upon a time there was
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
a
Enter your line: Once upon a time there was a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
little
Enter your line: We cannot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
have
Enter your line: 1
